#Facial Recognition
###A basic approach to facial recognition using the SVD

###The code
First and most importantly, import statements.

In [ ]:
%matplotlib inline
import numpy as np
from scipy import linalg as la
from os import walk
from scipy.ndimage import imread
from matplotlib import pyplot as plt
import matplotlib.cm as cm
import random

Below is the code to load the database of images, as well as some functions to show the images. 

In [ ]:
def getFaces(path='./faces94'):
    """Traverse the directory specified by 'path' and return an array containing
    one column vector per subdirectory.
    """
    # Traverse the directory and get one image per subdirectory
    faces = []
    for (dirpath, dirnames, filenames) in walk(path):
        for f in filenames:
            if f[-3:]=="jpg": # only get jpg images
                # load image, convert to grayscale, flatten into vector
                face = imread(dirpath+"/"+f).mean(axis=2).ravel()
                faces.append(face)
                break
    # put all the face vectors column-wise into a matrix
    return np.array(faces).T

def show(im, w=200, h=180):
    """Plot the flattened grayscale image 'im' of width 'w' and height 'h'."""
    plt.imshow(im.reshape((w,h)), cmap=cm.Greys_r)
    plt.axis("off")
    plt.show()
    
def show2(test_image, result, w=200, h=180):
    """Convenience function for plotting two flattened grayscale images of
    the specified width and height side by side
    """
    plt.subplot(121)
    plt.title("Inputed Image")
    plt.imshow(test_image.reshape((w,h)), cmap=cm.Greys_r)
    plt.axis("off")
    plt.subplot(122)
    plt.title("Closest Match")
    plt.imshow(result.reshape((w,h)), cmap=cm.Greys_r)
    plt.axis("off")
    plt.show()

We display one of the faces to make sure this is working correctly. There should be 153 in all.

In [ ]:
images = getFaces()
print("Number of faces: {}".format(images.shape[1]))
show(images[:,25])

The FacialRec class contains all of the methods and variables for our facial recognition system. We will run through each of the methods below.

In [ ]:
class FacialRec:
    
    def __init__(self, path):
        self.initFaces(path)
        self.initMeanImage()
        self.initDifferences()
        self.initEigenfaces()
        
    def initFaces(self, path):
        self.F = getFaces(path)
        
    def initMeanImage(self):
        self.mu = np.mean(self.F, axis=1)
        
    def initDifferences(self):
        self.Fbar = self.F - np.vstack(self.mu)
        
    def initEigenfaces(self):
        self.U, s, Vt = la.svd(self.Fbar, full_matrices = False)
        
    def project(self, A, s=38):
        return self.U[:,:s].T.dot(A)
    
    def findNearest(self, image, s=38):
        Fhat = self.U[:,:s].T.dot(self.Fbar)
        ghat = self.U[:,:s].T.dot(image - np.vstack(self.mu))
        return np.argmin(np.linalg.norm(Fhat - ghat, axis=0), axis=0)

###The mean face
After retrieving one face image of each person, the mean face is the average of all 153 faces. We create an instance of the FacialRec class and display the mean face below.

In [ ]:
facialRec = FacialRec('./faces94')
show(facialRec.mu)

###Shifting by the mean
This algorithm requires that the data be centered at zero. This can be thought of as accentuating the unique features of each face. To center the data at zero, we have to shift each of our sample faces by the mean face. Below is displayed the 25th face and the same face after shifting by the mean.

In [ ]:
plt.title("Original face")
show(facialRec.F[:,24])
plt.title("Mean-shifted")
show(facialRec.Fbar[:,24])

###Eigenfaces
We use the SVD to reduce the dimensionality of our data. By projecting to an $s$-dimensional subspace, instead of storing and comparing $200 \times 180$ values for each face image we only need $s$ values. 
The new subspace we project to is determined by the SVD of $\bar{F}$.
Basis vectors of this subspace are called "eigenfaces".
We represent the image by its coordinate vector - this contains the coefficients in the linear combination of eigenfaces that makes up this particular face.

 Below is displayed the first basis vector or eigenface.

In [ ]:
show(facialRec.U[:,0])

###Representing a face with fewer dimensions
We can project a face into the $s$ dimensional subspace, store it as a length-$s$ coordinate vector, and then "rebuild" it from the eigenfaces. Below we show the first face in the dataset rebuilt with $s = 5, 9, 19, 38,$ and $75$ eigenfaces. 

In [ ]:
for s in [5,9,19,38,75,153]:
    face = facialRec.Fbar[:,0]
    first_s = facialRec.project(face, s = s)
    standard = np.dot(facialRec.U[:,:s],first_s)
    result = standard + facialRec.mu
    plt.title("s = {}".format(s))
    show(result)

###Recognizing a face
To perform facial recognition, we project an unknown face image to the $s$ dimensional subspace. We compare its coordinate vector with the coordinate vectors of faces that we have already seen. We then return the closest match.

Below we randomly select 5 faces and find the closest match for each. We use $s = 38$ eigenfaces.

In [ ]:
def sampleFaces(n_tests, path="./faces94"):
    """Return an array containing a sample of n_tests images contained
    in the path as flattened images in the columns of the output
    """
    files = []
    for (dirpath, dirnames, filenames) in walk(path):
        for f in filenames:
            if f[-3:]=="jpg": # only get jpg images
                files.append(dirpath+"/"+f)

    #Get a sample of the images
    test_files = random.sample(files, n_tests)
    #Flatten and average the pixel values
    images = np.array([imread(f).mean(axis=2).ravel() for f in test_files]).T
    return images

for i in xrange(5):
    test_image = sampleFaces(1)    
    i = facialRec.findNearest(test_image, s=38)
    show2(test_image, facialRec.F[:,i])